In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/10.vehicle'):
    shutil.rmtree('./file/10.vehicle')

os.makedirs('./file/10.vehicle')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/9.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/9.part/{file}',
                           header=0,
                           dtype=str).fillna('')

    input_ = input_[input_['Vehicle Url'] != ''].reset_index(drop=True)
    
    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                df_temp, page = pd.DataFrame(), 1
                while True:
                    b = 0
                    while True:
                        b += 1

                        try:
                            resp = requests.get(f'''{input_.loc[a, 'Vehicle Url']}&start={page*50-50}&num=50''',
                                                headers=get_header(),
                                                proxies=get_proxy(),
                                                timeout=(10, 10))
                        
                            if resp.status_code == 200:
                                soup = BeautifulSoup(resp.text, 'lxml')
                                html = etree.HTML(str(soup))
                    
                                if html.xpath('//div[@class="detail-app-pager"]'):
                                    break
                        except KeyboardInterrupt:
                            break
                        except:
                            continue
                
                    # = = = = = = = = = = = = = = =
                
                    next_ = html.xpath('//a[@id="pagingBottom_nextButton"]')
                
                    # = = = = = = = = = = = = = = =
                
                    if next_ and next_[0].xpath('./@href')[0].strip() == 'Search/#':
                        raise
                        
                    # = = = = = = = = = = = = = = =
                
                    list_th = [text.strip() for text in html.xpath('//th[@scope="col"]/text()')]
                    
                    # = = = = = = = = = = = = = = =
                
                    list_tr = html.xpath('//tr[@class="detail-app-row"]')
                
                    # = = = = = = = = = = = = = = =
                
                    df_temp_temp = pd.DataFrame({'No': input_.loc[a, 'No'],
                                                 'Part Number': input_.loc[a, 'Part Number'],
                                                 'Vehicle Url': input_.loc[a, 'Vehicle Url'],
                                                 'Page': page,
                                                 'Row': [i+1 for i in range(len(list_tr))]})
                
                    for i, tr in zip(range(len(list_tr)), list_tr):
                        for j, th in zip(range(len(list_th)), list_th):
                            df_temp_temp.loc[i, th] = tr.xpath(f'./td[{j+1}]/text()')[0].strip()
                
                    df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')
                
                    # = = = = = = = = = = = = = = =

                    if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 100 == 0 and work.qsize() != 0:
                        clear_output()

                    print(f'''[尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} - {input_.loc[a, 'No']}.{input_.loc[a, 'Part Number']} > Page {page}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

                    # = = = = = = = = = = = = = = =
                
                    page += 1
                    
                    # = = = = = = = = = = = = = = =
                
                    if not next_:
                        break

                # = = = = = = = = = = = = = = =

                if df_temp.empty:
                    raise

                # = = = = = = = = = = = = = = =

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(30):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No', 'Page', 'Row'],
                                                    ascending=[True, True, True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/10.vehicle/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-vehicle_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/10.vehicle'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/10.vehicle/{file}',
              f'''./file/10.vehicle/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：1

[尝试次数：5] - 33.33% - crawler_695-vehicle_error - 46.918-975 > Page 1
[剩余数量：0] - [当前时间：08:39:13]

输出ing...

总数量：1

[尝试次数：1] - 66.67% - crawler_704-vehicle_error - 30.918-975 > Page 1
[剩余数量：0] - [当前时间：08:39:17]

输出ing...

总数量：1

[尝试次数：11] - 100.00% - crawler_902-vehicle_error - 2.918-975 > Page 1
[剩余数量：0] - [当前时间：08:39:19]

输出ing...



Progress: 100%|██████████████████████████████| 3/3 [00:00<00:00, 2659.11it/s]

Done ~
